In [4]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
from brush import BrushGauss
import numpy as np
from lmfit import Parameters, printfuncs
from refnx.analysis import Transform, CurveFitter
from refnx.dataset import ReflectDataset
%matplotlib inline
import matplotlib.pyplot as plt

In [6]:
# load the dataset

d = ReflectDataset('c_PLP0024650.dat')

In [7]:
t = Transform('logY').transform

In [8]:
# Transform the data to log R
d.y, d.y_err = t(d.x, d.y, d.y_err)

In [9]:
a = BrushGauss(0.46, 3, n_slices=25, dq=9, transform=t)

In [10]:
a.parameter_names()

['scale',
 'bkg',
 'SLD_super',
 'SLD_sub',
 'thickness_SiO2',
 'SLD_SiO2',
 'roughness_SiO2',
 'roughness_backing',
 'phi_init',
 'thickness_tail',
 'roughness_tail2int',
 'thickness_1',
 'phi_1',
 'roughness_1',
 'thickness_2',
 'phi_2',
 'roughness_2',
 'thickness_3',
 'phi_3',
 'roughness_3']

In [11]:
vals = [1, 1e-6, 2.07, 5.994, 8.8, 3.25, 3.5,5, 0.1, 424.2, 4,
            28, 0.95, 2, 50, 0.85, 2, 100, 0.2, 20]
lbs = [0.95, 1e-7, 2.06, 5.8, 8.7, 3.24, 3.4, 2, 0.006, 50,3,
            10, 0.1, 2, 10, 0.1, 2, 10, 0.1, 2]
ubs = [1.1, 1e-6, 2.08, 6.15, 8.9, 3.26, 3.6, 8, 0.5, 500,8,
            200, 0.8, 10, 200, 0.8, 200, 50, 0.8, 10]
varies = np.array([True] * 20, bool)
varies[[2,3,4,5,6]] = False

In [12]:
P = Parameters()

In [13]:
for name, val, vary, lb, ub in zip(a.parameter_names(), vals, varies, lbs, ubs):
    P.add(name, val, vary, lb, ub)

In [14]:
slab_pars = a.to_slab(P)

In [15]:
assert(slab_pars[0] * 4 + 8 == np.size(slab_pars))

In [16]:
z, rho = a.sld_profile(P)
z2, rho2 = a.vol_fraction(P)

TypeError: sld_profile() got an unexpected keyword argument 'z'

In [ ]:
P['roughness_1']

In [ ]:
plt.plot(z2,rho2)

In [ ]:
area = a.vfp_area(P)
print(area)

In [ ]:
q = np.linspace(0.01, 0.3, 1000)
ref = a.model(q, P)

In [ ]:
plt.plot(q, ref)
# plt.xscale('log')

In [ ]:
# b = CurveFitter(a, (q, ref), P)
# you can give CurveFitter a dataset
b = CurveFitter(a, d, P)

In [ ]:
c = b.fit('differential_evolution')

In [ ]:
print(printfuncs.fit_report(c))

In [ ]:
fit_ref = a.model(d.x,c.params)
plt.plot(d.x,d.y,'o',d.x,fit_ref,'-')

In [ ]:
c.params

In [ ]:
a.vfp_area(c.params)

In [ ]:
z, rho = a.vol_fraction(c.params)
plt.plot(z,rho)